In [1]:
import cspython.scraper
import pandas as pd

In [2]:
series = cspython.scraper.scrape_series_data('BIG', '2017-01-01', '2017-02-01')

https://www.hltv.org/results?offset=0&content=demo&team=7532&startDate=2017-01-01&endDate=2017-02-01
https://www.hltv.org/matches/2307616/big-vs-ination-igamecom-winter-invitational
https://www.hltv.org/matches/2307616/big-vs-ination-igamecom-winter-invitational
https://www.hltv.org/stats/matches/mapstatsid/41040/ination-vs-big
https://www.hltv.org/stats/matches/performance/mapstatsid/41040/ination-vs-big
match 0 done
https://www.hltv.org/matches/2307615/big-vs-igamecom-igamecom-winter-invitational
https://www.hltv.org/matches/2307615/big-vs-igamecom-igamecom-winter-invitational
https://www.hltv.org/stats/matches/mapstatsid/41037/big-vs-igamecom
https://www.hltv.org/stats/matches/performance/mapstatsid/41037/big-vs-igamecom
match 1 done
https://www.hltv.org/matches/2307440/flipsid3-vs-big-dreamhack-astro-open-leipzig-2017
https://www.hltv.org/matches/2307440/flipsid3-vs-big-dreamhack-astro-open-leipzig-2017
https://www.hltv.org/stats/matches/41029/flipsid3-vs-big
https://www.hltv.org/s

In [3]:
series[0].keys()

['url',
 'demo_url',
 'teams',
 'vetos',
 'match_data',
 'stats_url',
 'team_a_b',
 'match_info',
 'stats_data']

In [10]:
series[1]['teams'][0]

,BIG,K-D,+/-,ADR,KAST,Rating2.0
0,Johannes 'nex' Maget nex,24-11,13,111.5,75.0%,1.56
1,Johannes 'tabseN' Wodarz tabseN,16-18,-2,87.4,83.3%,1.23
2,Fatih 'gob b' Dayik gob b,20-15,5,77.1,62.5%,1.18
3,Kevin 'keev' Bartholomäus keev,17-13,4,70.3,70.8%,1.17
4,Nikola 'LEGIJA' Ninić LEGIJA,17-12,5,71.0,66.7%,1.16


Each element of series is a dictionary containing data 1 series played by the team. They are in reverse chornological order. The elements of each series dictionary are:

    url:        the url on hltv of the Overview of the entire series
    demo_url:   the url of the demo, hosted on hltv later
    stats_url:  a url containing more detailed data about the data 
                (the scrapped data from this page is contained in stats_data)
    teams:      a dataframe containing the overall stats for each team in the series
    vetos:      the vetos in chronological order
    match_info: score and map name for each match
    team_a_b:   this provides order to the teams and match_info scores 
    

In [5]:
def match_score_dataframe(series):
    score = pd.DataFrame(columns = ['map_name', 'winner', series['team_a_b'][0], 
                                    series['team_a_b'][1]], 
                         index=range(len(series['match_info'])))
    for idx, match in enumerate(series['match_info']):
        
        map_name = match['map_name']
        score_a = int(match['scores'][0])
        score_b = int(match['scores'][1])
        if score_a > score_b:
            winner = series['team_a_b'][0]
        elif score_a < score_b:
            winner = series['team_a_b'][1]
        else:
            winner = 'draw'
        score.loc[idx, :]  = map_name, winner, score_a, score_b
    
    return score

In [7]:
for s in series:
    print match_score_dataframe(s)
    print s['vetos']

  map_name   winner BIG iNation
0   Mirage  iNation  13      16
[u'1. iNation removed Cobblestone', u'2. BIG removed Cache', u'3. iNation removed Dust2', u'4. BIG removed Train', u'5. iNation removed Overpass', u'6. BIG removed Nuke', u'7. Mirage was left over']
  map_name winner BIG iGame.com
0    Dust2    BIG  16         8
[u'1. BIG removed Nuke', u'2. iGame.com removed Mirage', u'3. BIG removed Train', u'4. iGame.com removed Cache', u'5. BIG removed Overpass', u'6. iGame.com removed Cobblestone', u'7. Dust2 was left over']
  map_name    winner FlipSid3 BIG
0    Cache  FlipSid3       16  11
1    Train  FlipSid3       16  11
[u'1. BIG removed Nuke', u'2. FlipSid3 removed Dust2', u'3. BIG picked Cache', u'4. FlipSid3 picked Train', u'5. BIG removed Mirage', u'6. FlipSid3 removed Cobblestone', u'7. Overpass was left over']
      map_name  winner Heroic BIG
0        Train  Heroic     16   4
1        Dust2     BIG     14  16
2  Cobblestone     BIG      9  16
[u'1. Heroic removed Cache', u

In [14]:
print matches[0]['stats_data'][0].keys()
print matches[0]['match_data'][0].keys()

['team_scores', 'match_time', 'team_endings']
['first_kills', 'total_team_kda', 'who_kill_who', 'awp_kills']


In [11]:
pd.Series(matches[2]['stats_data'][1]['team_endings']['team_b'])

0              [7532]
1            [ct_win]
2            [ct_win]
3      [bomb_defused]
4      [emptyHistory]
5      [emptyHistory]
6      [emptyHistory]
7            [ct_win]
8      [emptyHistory]
9      [emptyHistory]
10     [emptyHistory]
11     [emptyHistory]
12     [emptyHistory]
13           [ct_win]
14           [ct_win]
15           [ct_win]
16            [t_win]
17    [bomb_exploded]
18            [t_win]
19     [emptyHistory]
20     [emptyHistory]
21    [bomb_exploded]
22     [emptyHistory]
23     [emptyHistory]
24     [emptyHistory]
25     [emptyHistory]
26     [emptyHistory]
27     [emptyHistory]
28     [emptyHistory]
29     [emptyHistory]
30     [emptyHistory]
dtype: object

In [20]:
def create_team_ending_df(match):
    team_a = pd.Series(match['stats_data']['team_endings']['team_a'])
    team_b = pd.Series(match['stats_data']['team_endings']['team_b'])
    return pd.concat([team_a, team_b], axis=1)

def create_team_score_df(match):
    team_a = pd.Series(match['stats_data']['team_scores']['team_a'])
    team_b = pd.Series(match['stats_data']['team_scores']['team_b'])
    return pd.concat([team_a, team_b], axis=1)

In [21]:
for match in matches:
    if match['stats_data']['team_scores']['team_a']

,0,1
0,[BIG],[iNation]
1,[1-0],[]
2,[2-0],[]
3,[3-0],[]
4,[4-0],[]
5,[],[4-1]
6,[],[4-2]
7,[],[4-3]
8,[],[4-4]
9,[],[4-5]
